In [118]:
# pip install selenium
# pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz
# pip install spacy==2.3.5
# pip install pyresparser

In [119]:
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import pickle
import nltk
import numpy as np

In [2]:
with open('NaiveBayes.pickle', 'rb') as efile:
    clf=pickle.load(efile)
with open('TFIDFVectorizer.pickle', 'rb') as efile:
    word_vectorizer=pickle.load(efile)
with open('LabelEncoder.pickle', 'rb') as efile:
    le=pickle.load(efile)

c:\users\ankur\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator MultinomialNB from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\users\ankur\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LabelBinarizer from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\users\ankur\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator OneVsRestClassifier from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\users\ankur\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimat

In [3]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('english')) #creating a set of stop words
def cleanResume(resumeText):
  
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    word_tokens = word_tokenize(resumeText) #Tokenize words
    filtered_sentence = [w for w in word_tokens if not w in stop_words] #Contains words other than stop words
    filtered_sentence = map(lambda x: lemmatizer.lemmatize(x), filtered_sentence)
    
    resumeText=' '.join(filtered_sentence)
    return resumeText

In [4]:
def largest_indices(ary, n):
    flat = ary.flatten()
    indices = np.argpartition(flat, -n)[-n:]
    indices = indices[np.argsort(-flat[indices])]
    return indices

In [5]:
def test(sent):
  sent=cleanResume(sent)
  WordFeatures = word_vectorizer.transform([sent])
  x=largest_indices(clf.predict_proba(WordFeatures), 3)
  recommendation=le.inverse_transform(x)
  return recommendation

In [6]:
from pyresparser import ResumeParser
def resume_test(path):
    data = ResumeParser(path).get_extracted_data()
    sentences=" "
    omit=["name", "email", "mobile_number", "no_of_pages", "total_experience"]
    for key, value in data.items():
          if value !=None and key not in omit: 
            if type(value)==list:
                  sentences= sentences +" "+ key + " " + " ".join(value)
            else:
                  sentences= sentences +" "+ key + " " + value
    professions= test(sentences)
    return (professions,data)
    

In [7]:
professions,data= resume_test('VithikaPungliya_Resume.pdf')

c:\users\ankur\appdata\local\programs\python\python39\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [8]:
import urllib.request as urllib2

In [9]:
print(professions)

['Data Science' 'Python Developer' 'Java Developer']


In [10]:
import random

In [13]:
# random.shuffle(professions)
# professions = ['Python Developer']
import requests

In [16]:
counter=0
l=[]
df = pd.DataFrame(columns=['Title','Company','Ratings','Salary','Location','Job_Post_History','URL', 'Skills', "Profession"])
HEADERS = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36\
                (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36"
            }
for i in professions:
    counter=counter+1
    for j in range(1,3):
        a=i.replace(" ","-")
        b=i.replace(" ","%20")
        url = f"https://www.naukri.com/{a}-jobs-{j}?k={b}"

#         driver = webdriver.Chrome("chromedriver.exe")

#         driver.get(url)
        r = requests.get(url,headers=HEADERS)
        time.sleep(1)

#         soup = BeautifulSoup(driver.page_source,'html5lib')
        print(r.text)
        soup = BeautifulSoup(r.text,"html.parser")

#         driver.close()

        results = soup.find(class_='list')
        job_elems = results.find_all('article',class_='jobTuple bgWhite br4 mb-8')

        for job_elem in job_elems:

            # URL to apply for the job     
            URL = job_elem.find('a',class_='title fw500 ellipsis').get('href')

            # Post Title
            Title = job_elem.find('a',class_='title fw500 ellipsis')
#             print("naukri title",Title.text)

            # Company Name
            Company = job_elem.find('a',class_='subTitle ellipsis fleft')
#             print("naukri company",Company.text)

            # Ratings
            rating_span = job_elem.find('span',class_='starRating fleft dot')
            if rating_span is None:
                pass
            else:
                Ratings = rating_span.text
#             print("naukri ratings",Ratings)

            # Salary offered for the job
            Sal = job_elem.find('li',class_='fleft grey-text br2 placeHolderLi salary')
            Sal_span = Sal.find('span',class_='ellipsis fleft fs12 lh16')
            if Sal_span is None:
                pass
            else:
                Salary = Sal_span.text

            # Location for the job post
            Loc = job_elem.find('li',class_='fleft grey-text br2 placeHolderLi location')
            Loc_exp = Loc.find('span',class_='ellipsis fleft fs12 lh16 locWdth')
            if Loc_exp is None:
                pass
            else:
                Location = Loc_exp.text

            # Number of days since job posted
            Hist = job_elem.find("div",["type br2 fleft grey","type br2 fleft green"])
            Post_Hist = Hist.find('span',class_='fleft fw500')
            if Post_Hist is None:
                pass
            else:
                Post_History = Post_Hist.text
                
            #Skills
#             skills=job_elem.find_all("ul", ["tags has-description"])
#             print("naukri skills",skills)
            for div in soup.find_all(class_='skill'):
                #print(div.text)
                skills=div.text

        #   Appending data to the DataFrame 
            
            l.append({'URL':URL,'Title':Title.text,'Company':Company.text,'Ratings':Ratings,'Location':Location,'Salary':Salary,'Job_Post_History':Post_History, "Profession":i,"Profession_key":counter, "Skills":skills})
            
#             vertical_concat = pd.concat([df1, df], axis=0)
#         print(df)
# print(l)
df = pd.DataFrame(l)

<HTML><HEAD>
<TITLE>Access Denied</TITLE>
</HEAD><BODY>
<H1>Access Denied</H1>
 
You don't have permission to access "http&#58;&#47;&#47;www&#46;naukri&#46;com&#47;Data&#45;Science&#45;jobs&#45;1&#63;" on this server.<P>
Reference&#32;&#35;18&#46;2d533617&#46;1667379138&#46;299e095f
</BODY>
</HTML>



AttributeError: 'NoneType' object has no attribute 'find_all'

In [149]:
#Display on Website
#Every Profession highest cosine similarity 10
#Profession
#Company
#Position
#Ratings- Filter
#Location
#Job_Post_History
#URL
#Skills
#Cosine Similiarty Index

#Index
#Resume Skills and Needed/Required Skills- Cosine Similarity

In [150]:
df

,URL,Title,Company,Ratings,Location,Salary,Job_Post_History,Profession,Profession_key,Skills
0,https://www.naukri.com/job-listings-assistant-...,Assistant Manager - Data Science,Citiustech,4.0,"Hybrid - Mumbai, Pune, Bangalore/Bengaluru",Not disclosed,1 Day Ago,Data Science,1,"[[[python], [spark], [kafka], [mobile applicat..."
1,https://www.naukri.com/job-listings-senior-ana...,"Senior Analyst, Data Science & Analytics",Transunion,4.3,Pune,Not disclosed,4 Days Ago,Data Science,1,"[[[python], [spark], [kafka], [mobile applicat..."
2,https://www.naukri.com/job-listings-data-scien...,Data Science/Analyst,Myfuturejob,4.3,New Delhi,Not disclosed,1 Day Ago,Data Science,1,"[[[python], [spark], [kafka], [mobile applicat..."
3,https://www.naukri.com/job-listings-data-scien...,Data Science Engineer 5.1,ANZ,4.1,Bangalore/Bengaluru,Not disclosed,1 Day Ago,Data Science,1,"[[[python], [spark], [kafka], [mobile applicat..."
4,https://www.naukri.com/job-listings-data-scien...,Data Science Engineer 4.4,ANZ,4.1,Bangalore/Bengaluru,Not disclosed,1 Day Ago,Data Science,1,"[[[python], [spark], [kafka], [mobile applicat..."
...,...,...,...,...,...,...,...,...,...,...
74,https://www.naukri.com/job-listings-java-devel...,Java Developer - Godrej Infotech Ltd,Godrej Infotech,4.0,"Hybrid - Pune, Bangalore/Bengaluru, Delhi / NC...",Not disclosed,1 Day Ago,Java Developer,3,"[[[python], [spark], [kafka], [mobile applicat..."
75,https://www.naukri.com/job-listings-java-devel...,Java Developer,Mlogica,4.0,Pune,Not disclosed,1 Day Ago,Java Developer,3,"[[[python], [spark], [kafka], [mobile applicat..."
76,https://www.naukri.com/job-listings-java-devel...,Java Developer,Barnwal Consultancy And Services,4.0,Pune,Not disclosed,1 Day Ago,Java Developer,3,"[[[python], [spark], [kafka], [mobile applicat..."
77,https://www.naukri.com/job-listings-core-java-...,Core Java Developer For Reputed MNC Client-Imm...,The It Mind Services,4.6,"Pune, Bangalore/Bengaluru, Delhi / NCR","50,000 - 1,00,000 PA.",Just Now,Java Developer,3,"[[[python], [spark], [kafka], [mobile applicat..."


In [159]:
df["Skills"]

0     [[[python], [spark], [kafka], [mobile applicat...
1     [[[python], [spark], [kafka], [mobile applicat...
2     [[[python], [spark], [kafka], [mobile applicat...
3     [[[python], [spark], [kafka], [mobile applicat...
4     [[[python], [spark], [kafka], [mobile applicat...
                            ...                        
74    [[[python], [spark], [kafka], [mobile applicat...
75    [[[python], [spark], [kafka], [mobile applicat...
76    [[[python], [spark], [kafka], [mobile applicat...
77    [[[python], [spark], [kafka], [mobile applicat...
78    [[[python], [spark], [kafka], [mobile applicat...
Name: Skills, Length: 79, dtype: object

In [160]:
df["Skills"].loc[1]

[<ul class="tags has-description"><li class="fleft fs12 grey-text lh16 dot">python</li><li class="fleft fs12 grey-text lh16 dot">spark</li><li class="fleft fs12 grey-text lh16 dot">kafka</li><li class="fleft fs12 grey-text lh16 dot">mobile application design</li><li class="fleft fs12 grey-text lh16 dot">web services</li><li class="fleft fs12 grey-text lh16 dot">solr</li><li class="fleft fs12 grey-text lh16 dot">scalability</li></ul>]

In [161]:
#Build Cosine Similarity

In [162]:
def convert_list(ls):
    ls=np.array(ls)
    l=[]
    return ls.flatten()

In [163]:
import itertools

In [164]:
df["Skills"]=df["Skills"].apply(convert_list)

In [171]:
df["Skills"]

0     [python, spark, kafka, mobile application desi...
1     [python, spark, kafka, mobile application desi...
2     [python, spark, kafka, mobile application desi...
3     [python, spark, kafka, mobile application desi...
4     [python, spark, kafka, mobile application desi...
                            ...                        
74    [python, spark, kafka, mobile application desi...
75    [python, spark, kafka, mobile application desi...
76    [python, spark, kafka, mobile application desi...
77    [python, spark, kafka, mobile application desi...
78    [python, spark, kafka, mobile application desi...
Name: Skills, Length: 79, dtype: object

In [172]:
df

,URL,Title,Company,Ratings,Location,Salary,Job_Post_History,Profession,Profession_key,Skills
0,https://www.naukri.com/job-listings-assistant-...,Assistant Manager - Data Science,Citiustech,4.0,"Hybrid - Mumbai, Pune, Bangalore/Bengaluru",Not disclosed,1 Day Ago,Data Science,1,"[python, spark, kafka, mobile application desi..."
1,https://www.naukri.com/job-listings-senior-ana...,"Senior Analyst, Data Science & Analytics",Transunion,4.3,Pune,Not disclosed,4 Days Ago,Data Science,1,"[python, spark, kafka, mobile application desi..."
2,https://www.naukri.com/job-listings-data-scien...,Data Science/Analyst,Myfuturejob,4.3,New Delhi,Not disclosed,1 Day Ago,Data Science,1,"[python, spark, kafka, mobile application desi..."
3,https://www.naukri.com/job-listings-data-scien...,Data Science Engineer 5.1,ANZ,4.1,Bangalore/Bengaluru,Not disclosed,1 Day Ago,Data Science,1,"[python, spark, kafka, mobile application desi..."
4,https://www.naukri.com/job-listings-data-scien...,Data Science Engineer 4.4,ANZ,4.1,Bangalore/Bengaluru,Not disclosed,1 Day Ago,Data Science,1,"[python, spark, kafka, mobile application desi..."
...,...,...,...,...,...,...,...,...,...,...
74,https://www.naukri.com/job-listings-java-devel...,Java Developer - Godrej Infotech Ltd,Godrej Infotech,4.0,"Hybrid - Pune, Bangalore/Bengaluru, Delhi / NC...",Not disclosed,1 Day Ago,Java Developer,3,"[python, spark, kafka, mobile application desi..."
75,https://www.naukri.com/job-listings-java-devel...,Java Developer,Mlogica,4.0,Pune,Not disclosed,1 Day Ago,Java Developer,3,"[python, spark, kafka, mobile application desi..."
76,https://www.naukri.com/job-listings-java-devel...,Java Developer,Barnwal Consultancy And Services,4.0,Pune,Not disclosed,1 Day Ago,Java Developer,3,"[python, spark, kafka, mobile application desi..."
77,https://www.naukri.com/job-listings-core-java-...,Core Java Developer For Reputed MNC Client-Imm...,The It Mind Services,4.6,"Pune, Bangalore/Bengaluru, Delhi / NCR","50,000 - 1,00,000 PA.",Just Now,Java Developer,3,"[python, spark, kafka, mobile application desi..."


In [173]:
user_skills=" ".join(data["skills"])
user_skills

'Matplotlib Opencv Data analytics Analysis Mining Tensorflow Pandas Math Python Compliance Communication Numpy Parser Database Java Architecture Audio Analytics Website Video Linux Cloud Machine learning Programming C Technical Forecasting Github Research Green Ai Sql Keras'

In [174]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
cv = CountVectorizer()
def score(ls):
    JobText=' '.join(ls)
    text=[user_skills,JobText]
    count_matrix = cv.fit_transform(text)
    matchPercentage=cosine_similarity(count_matrix)[0][1] * 100
    return round(matchPercentage, 2)
    

In [175]:
def score(ls,user_skills):
    cv = CountVectorizer()
    JobText=' '.join(ls)
    text=[user_skills,JobText]
    count_matrix = cv.fit_transform(text)
    matchPercentage=cosine_similarity(count_matrix)[0][1] * 100
    return round(matchPercentage, 2)

In [176]:
user_skills=" ".join(data["skills"])
df["score"]=df["Skills"].apply(lambda x: score(x,user_skills))

In [177]:
df["score"]

0     5.27
1     5.27
2     5.27
3     5.27
4     5.27
      ... 
74    5.27
75    5.27
76    5.27
77    5.27
78    5.27
Name: score, Length: 79, dtype: float64

In [179]:
# df["score"]=df["Skills"].apply(score)

In [180]:
top15=df.sort_values('score',ascending=False)[:15]

In [181]:
top15

,URL,Title,Company,Ratings,Location,Salary,Job_Post_History,Profession,Profession_key,Skills,score
0,https://www.naukri.com/job-listings-assistant-...,Assistant Manager - Data Science,Citiustech,4.0,"Hybrid - Mumbai, Pune, Bangalore/Bengaluru",Not disclosed,1 Day Ago,Data Science,1,"[python, spark, kafka, mobile application desi...",5.27
50,https://www.naukri.com/job-listings-python-dev...,PYTHON Developer,Mplussoft India,3.8,Pune,Not disclosed,12 Days Ago,Python Developer,2,"[python, spark, kafka, mobile application desi...",5.27
57,https://www.naukri.com/job-listings-python-dev...,Python Developer,RawData Technologies,4.1,"Kolkata, Mumbai, Hyderabad/Secunderabad, Pune,...",Not disclosed,Few Hours Ago,Python Developer,2,"[python, spark, kafka, mobile application desi...",5.27
56,https://www.naukri.com/job-listings-sr-python-...,Sr. Python Developer: Immediate Joiner,Fhirengine India,4.1,Permanent Remote,Not disclosed,3 Days Ago,Python Developer,2,"[python, spark, kafka, mobile application desi...",5.27
55,https://www.naukri.com/job-listings-mindtree-i...,Mindtree is looking For Python developer,Mindtree,4.1,"Temp. WFH - Kolkata, Hyderabad/Secunderabad, P...",Not disclosed,13 Days Ago,Python Developer,2,"[python, spark, kafka, mobile application desi...",5.27
54,https://www.naukri.com/job-listings-senior-pyt...,Senior Python Developer,Chateaux,4.6,Temp. WFH - Bangalore/Bengaluru,Not disclosed,1 Day Ago,Python Developer,2,"[python, spark, kafka, mobile application desi...",5.27
53,https://www.naukri.com/job-listings-senior-pyt...,Senior Python Developer - Django / Flask,BootLabs,4.6,Bangalore/Bengaluru,Not disclosed,1 Day Ago,Python Developer,2,"[python, spark, kafka, mobile application desi...",5.27
52,https://www.naukri.com/job-listings-python-pro...,Python Programming Language Application Developer,Accenture,4.2,Bangalore/Bengaluru,Not disclosed,1 Day Ago,Python Developer,2,"[python, spark, kafka, mobile application desi...",5.27
51,https://www.naukri.com/job-listings-hiring-pyt...,Hiring Python developer,Indecomm,3.7,"Chennai, Bangalore/Bengaluru",Not disclosed,14 Days Ago,Python Developer,2,"[python, spark, kafka, mobile application desi...",5.27
49,https://www.naukri.com/job-listings-python-dev...,Python Developer,Vconstruct,4.3,Pune,Not disclosed,1 Day Ago,Python Developer,2,"[python, spark, kafka, mobile application desi...",5.27


In [28]:
df_sorted=df.sort_values('score',ascending=False)

In [29]:
top5_2 = df_sorted[df_sorted['Profession_key'] == 2.0 ][:5]

KeyError: 'Profession_key'

In [ ]:
top5_2

In [ ]:
top5_3=df_sorted[df_sorted['Profession_key'] == 3.0 ][:5]

In [ ]:
top5_3